In [46]:
import requests
import numpy as np
import io
import pandas as pd


In [97]:
response = requests.get("https://mercados.ambito.com//dolar/oficial/historico-general/01-12-2013/31-12-2018")

In [98]:
response

<Response [200]>

In [99]:
type(response.content)

bytes

In [100]:
array = np.asarray(io.BytesIO(response.content))

In [101]:
str_content = response.content.decode("utf-8")

In [102]:
str_content_list = str_content.split("],[")

In [103]:
len(str_content_list)

1241

In [104]:
str_content_list[:10]

['[["Fecha","Compra","Venta"',
 '"28-12-2018","36,83","38,83"',
 '"27-12-2018","37,41","39,41"',
 '"26-12-2018","37,48","39,48"',
 '"21-12-2018","37,02","39,02"',
 '"20-12-2018","37,04","39,04"',
 '"19-12-2018","37,33","39,33"',
 '"18-12-2018","37,20","39,20"',
 '"17-12-2018","37,25","39,25"',
 '"14-12-2018","37,16","39,16"']

In [105]:
def parse_content(contenido):
    contenido = contenido.replace('","',';')
    contenido = contenido.replace('"','')
    contenido = contenido.replace(',','.')
    contenido = contenido.replace('[','')
    contenido = contenido.replace(']','')
    return contenido

In [106]:
lista_final = [parse_content(contenido).split(";") for contenido in str_content_list]

In [107]:
headers = lista_final.pop(0)

In [108]:
headers

['Fecha', 'Compra', 'Venta']

In [109]:
cotizacion_df = pd.DataFrame(lista_final, columns = headers)

In [110]:
cotizacion_df.dtypes

Fecha     object
Compra    object
Venta     object
dtype: object

In [111]:
cotizacion_df.Fecha = pd.to_datetime(cotizacion_df.Fecha, format="%d-%m-%Y")

In [112]:
cotizacion_df["anio"] = cotizacion_df.Fecha.dt.year
cotizacion_df["mes"] = cotizacion_df.Fecha.dt.month
cotizacion_df["dia"] = cotizacion_df.Fecha.dt.day

In [121]:
cotizacion_df.Compra = pd.to_numeric(cotizacion_df.Compra)
cotizacion_df.Venta = pd.to_numeric(cotizacion_df.Venta)

In [122]:
min_days = cotizacion_df[["anio","mes","dia"]].groupby(["anio", "mes"]).min().reset_index()

In [140]:
cotizacion_mensual = pd.merge(min_days,
                                cotizacion_df,
                                left_on = ["anio","mes","dia"],
                                right_on = ["anio","mes","dia"],
                                how = "left")

In [141]:
cotizacion_mensual["cotizacion"] = (cotizacion_mensual.Compra + cotizacion_mensual.Venta) /2

In [142]:
cotizacion_mensual = cotizacion_mensual[["anio", "mes", "cotizacion"]]

In [143]:
cotizacion_mensual["cotizacion_mes_anterior"] = cotizacion_mensual.cotizacion.shift(1)

In [144]:
cotizacion_mensual = cotizacion_mensual.dropna()

In [146]:
cotizacion_mensual["aumento"] = cotizacion_mensual.cotizacion/cotizacion_mensual.cotizacion_mes_anterior - 1

In [151]:
cotizacion_mensual.anio = cotizacion_mensual.anio.map(str)
cotizacion_mensual.mes = cotizacion_mensual.mes.map(str)
cotizacion_mensual.mes = cotizacion_mensual.mes.str.pad(width=2, side="left", fillchar="0")

In [153]:
cotizacion_mensual = cotizacion_mensual[["mes","anio","cotizacion","aumento"]]

In [154]:
cotizacion_mensual.to_csv("../../data/processed/cotizacion_dolar.csv", index=False)